In [2]:
import numpy as np
from tempor import plugin_loader

In [3]:
print(plugin_loader.list())

{'prediction': {'one_off': {'classification': ['nn_classifier', 'cde_classifier', 'laplace_ode_classifier', 'ode_classifier'], 'regression': ['ode_regressor', 'nn_regressor', 'laplace_ode_regressor', 'cde_regressor']}, 'temporal': {'classification': ['seq2seq_classifier'], 'regression': ['seq2seq_regressor']}}, 'preprocessing': {'encoding': {'static': ['static_onehot_encoder'], 'temporal': ['ts_onehot_encoder']}, 'imputation': {'static': ['static_tabular_imputer'], 'temporal': ['bfill', 'ts_tabular_imputer', 'ffill']}, 'nop': ['nop_transformer'], 'scaling': {'static': ['static_standard_scaler', 'static_minmax_scaler'], 'temporal': ['ts_standard_scaler', 'ts_minmax_scaler']}}, 'time_to_event': ['ts_coxph', 'ts_xgb', 'dynamic_deephit'], 'treatments': {'one_off': {'regression': ['synctwin_regressor']}, 'temporal': {'classification': ['crn_classifier'], 'regression': ['crn_regressor']}}}


In [4]:
# Load a dataset with temporal treatments and outcomes:
dataset = plugin_loader.get(
    "treatments.temporal.dummy_treatments",
    plugin_type="datasource",
    temporal_covariates_missing_prob=0.0,
    temporal_treatments_n_features=1,
    temporal_treatments_n_categories=2,
).load()

In [5]:
dataset

TemporalTreatmentEffectsDataset(
    time_series=TimeSeriesSamples([100, *, 5]),
    static=StaticSamples([100, 3]),
    predictive=TemporalTreatmentEffectsTaskData(
        targets=TimeSeriesSamples([100, *, 2]),
        treatments=TimeSeriesSamples([100, *, 1])
    )
)

In [6]:
dataset.time_series

TimeSeriesSamples with data:
                             0          1          2          3          4
sample_idx time_idx                                                       
0          0          0.419045   0.893763   0.780285   0.660508   1.047522
           1          1.257931   2.172271   2.226089   2.360713   1.981578
           2          2.247657   0.853397   2.525946   3.213647   2.897191
           3          3.396456   5.386071   3.721545   2.503248   3.517212
           4          4.387812   3.365264   5.612532   5.573375   4.767746
...                        ...        ...        ...        ...        ...
99         12        12.654769  14.810888  12.914859  11.697631  12.818675
           13        13.418815  12.135655  12.481295  13.336797  13.696168
           14        13.785503  14.431228  15.193174  17.551818  14.464249
           15        15.344934  15.916966  14.368132  15.965113  15.419334
           16        16.033907  15.162631  17.338485  17.007235  17.034645

[1547 rows x 5 columns]

In [7]:
dataset.static

,0,1,2
sample_idx,,,
0,0.753423,3.239284,0.995587
1,0.829240,3.175298,0.770566
2,0.674581,3.229741,1.302317
3,0.584040,3.234011,1.594861
4,0.501552,3.211027,0.639503
...,...,...,...
95,0.680235,3.287749,0.705369
96,0.788814,3.313229,1.318394
97,0.589116,3.268607,1.646737


In [8]:
dataset.predictive.treatments[0]

TimeSeriesSamples with data:
                     0
sample_idx time_idx   
0          0         1
           1         0
           2         1
           3         0
           4         0
           5         1
           6         1
           7         1
           8         1
           9         0
           10        1
           11        0
           12        1
           13        1
           14        0
           15        0
           16        0

In [12]:
dataset.predictive.targets.dataframe().head(100)

0         1
sample_idx time_idx                    
0          0        -1.433570  0.714861
           1        -0.600733  2.744446
           2         0.622874  1.816995
           3         1.879785  4.981217
           4         2.477957  5.932101
...                       ...       ...
6          2         0.514816  2.072122
           3         1.443873  3.482707
           4         2.444069  5.764491
           5         3.585320  5.160905
           6         4.506359  6.420430

[100 rows x 2 columns]

In [10]:
# Initialize the model:
model = plugin_loader.get("treatments.temporal.regression.crn_regressor", epochs=20)

In [11]:
# Train:
model.fit(dataset)

Preparing data for decoder training...
Preparing data for decoder training DONE.
=== Training stage: 1. Train encoder ===
Epoch: 0, Prediction Loss: 76.517, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 76.517
Epoch: 1, Prediction Loss: 38.771, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 38.771
Epoch: 2, Prediction Loss: 20.237, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 20.237
Epoch: 3, Prediction Loss: 15.108, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 15.108
Epoch: 4, Prediction Loss: 15.413, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 15.413
Epoch: 5, Prediction Loss: 13.845, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 13.845
Epoch: 6, Prediction Loss: 10.080, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 10.080
Epoch: 7, Prediction Loss: 6.351, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 6.351
Epoch: 8, Prediction Loss: 4.483, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 4.483
Epoch: 9, Prediction Loss: 3.182, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 

CRNTreatmentsRegressor(
    name='crn_regressor',
    category='treatments.temporal.regression',
    plugin_type='method',
    params={
        'encoder_rnn_type': 'LSTM',
        'encoder_hidden_size': 100,
        'encoder_num_layers': 1,
        'encoder_bias': True,
        'encoder_dropout': 0.0,
        'encoder_bidirectional': False,
        'encoder_nonlinearity': None,
        'encoder_proj_size': None,
        'decoder_rnn_type': 'LSTM',
        'decoder_hidden_size': 100,
        'decoder_num_layers': 1,
        'decoder_bias': True,
        'decoder_dropout': 0.0,
        'decoder_bidirectional': False,
        'decoder_nonlinearity': None,
        'decoder_proj_size': None,
        'adapter_hidden_dims': [50],
        'adapter_out_activation': 'Tanh',
        'predictor_hidden_dims': [],
        'predictor_out_activation': None,
        'max_len': None,
        'optimizer_str': 'Adam',
        'optimizer_kwargs': {'lr': 0.01, 'weight_decay': 1e-05},
        'batch_size': 3

In [13]:
dataset = dataset[:5]

In [15]:
# Define horizons for each sample.
horizons = [tc.time_indexes()[0][len(tc.time_indexes()[0]) // 2 :] for tc in dataset.time_series]
print("Horizons for sample 0:\n", horizons[0], end="\n\n")

Horizons for sample 0:
 [8, 9, 10, 11, 12, 13, 14, 15, 16]


In [21]:
dataset.time_series[0].time_indexes()[0]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [35]:
# Define treatment scenarios for each sample.
treatment_scenarios = [[np.asarray([2] * len(h)), np.asarray([0] * len(h))] for h in horizons]
print("Alternative treatment scenarios for sample 0:\n", treatment_scenarios[0], end="\n\n")


Alternative treatment scenarios for sample 0:
 [array([2, 2, 2, 2, 2, 2, 2, 2, 2]), array([0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [36]:
# Call predict_counterfactuals.
counterfactuals = model.predict_counterfactuals(dataset, horizons=horizons, treatment_scenarios=treatment_scenarios)
print("Counterfactual outcomes for sample 0, given the alternative treatment scenarios:\n")
for idx, c in enumerate(counterfactuals[0]):
    print(f"Treatment scenario {idx}, {treatment_scenarios[0][idx]}")
    print(c, end="\n\n")

Counterfactual outcomes for sample 0, given the alternative treatment scenarios:

Treatment scenario 0, [2 2 2 2 2 2 2 2 2]
TimeSeries() with data:
                 0          1
time_idx                     
8         7.623644  10.405146
9         7.915308  10.677928
10        7.957008  10.714924
11        7.962815  10.720039
12        7.963622  10.720748
13        7.963733  10.720847
14        7.963748  10.720860
15        7.963750  10.720861
16        7.963750  10.720861

Treatment scenario 1, [0 0 0 0 0 0 0 0 0]
TimeSeries() with data:
                 0          1
time_idx                     
8         7.578797  10.197225
9         7.860027  10.441121
10        7.902677  10.479163
11        7.909140  10.484888
12        7.910116  10.485751
13        7.910263  10.485883
14        7.910285  10.485902
15        7.910288  10.485904
16        7.910289  10.485906


In [12]:
counterfactuals[0][0]

,0,1
time_idx,,
8,7.470183,10.196918
9,7.948002,10.625115
10,8.118283,10.779013
11,8.181903,10.836723
12,8.206075,10.858680
13,8.215317,10.867080
14,8.218859,10.870299
15,8.220217,10.871533
16,8.220738,10.872007
